In [ ]:
import binascii
import pickle
from collections import Counter
from heapq import heapify, heappush, heappop
import os
import time
 
 
global count, _elements, mylist, myroot,file,action
heap = []
heapify(heap)
 
 
res = []
print_array = ""
encoded_letters = []
encoded_string = ""
 
Huffman_tree = None
text = ""
Huffman_Codes = {}  # dictionary
 
reverse_Huffman = {}
code = ""
header = ""
padded_text_string = ""
 
 
# CREATING HUFFMAN TREE
def create_tree(myheap):
    while (len(myheap) > 1):
        left = heappop(myheap)
        right = heappop(myheap)
        val = left.getF() + right.getF()
        newnode = Node(val, None, left, right)
        heappush(myheap, newnode)
    Encode(myheap)
 
 
# PUSHING INTO HEAP FROM TEXT FILE
 
def push_into_heap(mylist):
    list_values = list(mylist.values())
    list_keys = list(mylist.keys())
    for i in range(len(mylist)):
        newnode = Node(list_values[i], list_keys[i], None, None)
        heappush(heap, newnode)
    create_tree(heap)
 
def PostorderTraversal(root):
    global res
    if root is not None:
        PostorderTraversal(root.left_child)
        PostorderTraversal(root.right_child)
        res.append(root.letter)
 
 
def myprint(myheap1):
    print("The heap elements : ")
    for i in myheap1:
        print(i.frequency)
        print("left_child: ", i.left_child.frequency)
        print("right_child: ", i.right_child.frequency)
 
 
 
def traverse_huffman_tree(root, print_array):  # TRAVERSING TREE
    global encoded_letters
    if root is None:
        return
 
    if root.letter is not None:
        encoded_letters.append(print_array)
        return
 
    traverse_huffman_tree(root.left_child, print_array + "0")
    traverse_huffman_tree(root.right_child, print_array + "1")
 
 
def store_huffman_codes(root, code):  #FUNCTION TO STORE HUFFMAN CODE IN A DICTIONARY
    traverse_huffman_tree(Huffman_tree, print_array) #BUILD TREE
    global Huffman_Codes, reverse_Huffman
    if root is None:
        return
 
    if root.letter is not None:
        Huffman_Codes[root.letter] = code
        reverse_Huffman[code] = root.letter
 
    store_huffman_codes(root.left_child, code + "0")
    store_huffman_codes(root.right_child, code + "1")
 
 
def decode_text(encoded_text,reversed):  #FUNCTION TO DECODE HUFFMAN CODE
    current_code = ""
    decoded_text = ""
    for bit in encoded_text:
 
        current_code += bit
        if current_code in reversed:
            character = reversed[current_code]
            decoded_text += character
            current_code = ""
    return decoded_text
 
 
def decode_huffman(root, encoded_string):
    answer = ""
    current = root
    for i in range(len(encoded_string)):
        if encoded_string[i] == '0':
            current = current.left_child
        else:
            current = current.right_child
 
        if current.right_child is None and current.left_child is None:
            answer += current.letter
            current = root
 
    return answer
 
def get_encoded_string():  #FUNCTION TO GET ENCODED STRING
    global encoded_string
    global Huffman_Codes, text
    for i in text:
        encoded_string += Huffman_Codes[i]
 
 
# NODE CLASS
 
class Node:
 
    def __init__(self, frequency, letter, left_child, right_child):
        self.letter = letter
        self.frequency = frequency
        self.left_child = left_child
        self.right_child = right_child
 
    def __lt__(self, other):
        return self.frequency < other.frequency
 
    def __str__(self):
        return str("{}:{}:{}:{}".format(self.frequency, self.letter, self.left_child, self.right_child))
 
    def getF(self):
        return self.frequency
 
 
stack = []
 
def pad_encoded_string(encoded): #PADDING STRING
 
    extra = 8 - len(encoded) % 8
    for i in range(extra):
        encoded += "0"
 
    padded_info = "{0:08b}".format(extra)
    encoded = padded_info + encoded
    return encoded
 
 
def get_byte_array(encoded): #BYTE ARRAY TO PUT INTO BINARY FILE
 
    if len(encoded) % 8 != 0:
        print("Encoded text not padded properly")
        exit(0)
 
    b = bytearray()
    for i in range(0, len(encoded), 8):
        b.append(int(encoded[i:i + 8], 2))
    return b
 
 
def Encode(myheap):
    global Huffman_tree, myroot, padded_text_string, reverse_Huffman, Huffman_Codes
    Huffman_tree = heappop(myheap)
 
    store_huffman_codes(Huffman_tree, code)  # BUILDS TREE AND STORES CODE
 
    get_encoded_string()  # GET ENCODED STRING AS A WHOLE
    x = file.split(".")
 
    output_path = x[0] + "_compressed.bin"
    padded_text_string = pad_encoded_string(encoded_string)
    padded_text_in_byte_array = get_byte_array(padded_text_string) # PADS STRING AND GETS ARRAY OF BYTES
 
 
    t1 = time.time()
    with open(output_path, "wb") as output_compressed_file:
 
        pickle.dump(reverse_Huffman, output_compressed_file) #SENDS DICTIONARY INTO BINARY FILE
        output_compressed_file.write(bytes(padded_text_in_byte_array)) #WRITE BYTE ARRAY INTO BINARY FILE
        _c = os.path.getsize(output_path)
        _o = os.path.getsize(file)
 
        t2 = time.time()
        print("file compressed!")
        print("original file size :", _o, "bytes")
        print("file size :", _c, "bytes")
        print("Compression Ratio :", _c/_o)
        print("Time Taken",t2-t1)
 
 
def compression():
    global text
    test = ""
    with open(file, 'r+') as info:
        text = info.read()
        text = text.rstrip()
        count = Counter(text)
        push_into_heap(count) #creates tree
 
 
def remove_padding(padded_encoded_text):  #REMOVES PADDING FOR DECODING
    padded_info = padded_encoded_text[:8]
    extra_padding = int(padded_info, 2)
 
 
    padded_encoded_text = padded_encoded_text[8:]
    encoded_text = padded_encoded_text[:-1 * extra_padding]
 
    return encoded_text
 
 
def decompression():
    x = file.split(".")
    decommpressed_file =x[0]+"_decompressed.txt"
    with open(file, 'rb') as compressed_file, open(decommpressed_file, 'w') as output:
        bit_string = ""
        inverse_code = pickle.load(compressed_file)
        byte_ = compressed_file.read(1)
        while byte_:
            byte_ = ord(byte_)
            bits = bin(byte_)[2:].rjust(8, '0')
            bit_string += bits
            byte_ = compressed_file.read(1)
 
        encoded_text_without_padding = remove_padding(bit_string)
        decompressed_text = decode_text(encoded_text_without_padding, inverse_code)
        output.write(decompressed_text) #WRITE INTO FILE
        print(decompressed_text)
        print("Decompressed!")
        _c = os.path.getsize(decommpressed_file)
        print("File Size: ", _c, "bytes")
 
    if action == "df":
        split()
 
 
numoffiles=0;
 
def compress_folder():
    global ext
    global numoffiles
    list = os.listdir(ext)  # dir is your directory path
    numoffiles = len(list)
    print("num of files:",numoffiles)
    ext = "/" + ext
    with open("folder.txt","w") as total:
        total.write(str(numoffiles))
        total.write("\n")
        for files_insides in os.listdir(os.getcwd()+ext):
            with open(os.path.join(os.getcwd()+ext, files_insides), "r") as f:
                total.write(str(sum(1 for _ in f)))
                total.write("\n")
 
        for files_insides in os.listdir(os.getcwd() + ext):
            with open(os.path.join(os.getcwd() + ext, files_insides), "r") as f:
                    total.write(f.read())
                    total.write("\n")
        global file
    file = "folder.txt"
    compression()
    os.remove("folder.txt")
 
 
 
 
def split():
    newpath = r'my_folder'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    smallfile = ""
    with open("folder_compressed_decompressed.txt") as bigfile:
        i = 1
        total_files = int(bigfile.readline())
        mynum = []
        for r in range(total_files):
            mynum.append(int(bigfile.readline()))
        for x in range(total_files):
            readlines = mynum[x]
            small_filename = 'my_folder/file_{}.txt'.format(i)
            i += 1
            smallfile = open(small_filename, "w")
            count =1
            for v in range(readlines):
                read = bigfile.readline()
                smallfile.write(read)
 
 
    os.remove("folder_compressed_decompressed.txt")
 
action = input("Enter the action you want to perform : ")
file = input("Enter text file: ")
 
if action[0] == "c" or action[0]=="C":
    compression()
elif action[0] == "d" or action[0]=="D":
    decompression()
elif action == "f":
    ext = file
    compress_folder()
elif action == "df":
    decompression()
 
else:
    print("Invalid Action")